In [8]:
import pandas as pd
import numpy as np
import pickle

In [9]:
# 1638 words
with open("../Data/wordpools/wasnorm_wordpool.txt") as f:
    items = f.readlines()
items = [i.replace("\n","") for i in items]
items = np.array(items)

In [10]:
simu_sess_num = 1000
n = 12
wordpool = np.arange(1,1639)
pos_rep = [1,3,5]

In [11]:
df_study = pd.DataFrame()
df_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for sess in range(simu_sess_num):
    tmp_df_study = pd.DataFrame()
    tmp_df_test1 = pd.DataFrame()
    tmp_df_test2 = pd.DataFrame()
    
    # choose present words
    pres_words = rng.choice(wordpool, 2*n, replace = False)
    pres_words = np.reshape(pres_words, (n,2))
    pair_idx = np.arange(1,13) + sess * n
    pres_items = pres_words
    
    tmp_df_study['study_itemno1'] = pres_items[:,0]
    tmp_df_study['study_itemno2'] = pres_items[:,1]
    tmp_df_study['study_item1'] = items[pres_items[:,0]-1]
    tmp_df_study['study_item2'] = items[pres_items[:,1]-1]
    tmp_df_study['session'] = sess
    df_study = pd.concat([df_study, tmp_df_study])
    
    # choose test words
    test_order = rng.permutation([[1,1]] * int(n/4) + [[1,2]] * int(n/4) + [[2,1]] * int(n/4) + [[2,2]] * int(n/4))
    test1_order = test_order[:,0]
    test2_order = test_order[:,1]
    pres_words = np.array(pres_words)
    test1_probe = np.array([pres_words[i,test1_order[i]-1] for i in range(n)])
    test1_ans = np.array([pres_words[i,2-test1_order[i]] for i in range(n)])
    test2_probe = np.array([pres_words[i,test2_order[i]-1] for i in range(n)])
    test2_ans = np.array([pres_words[i,2-test2_order[i]] for i in range(n)])
    
    # test
    probe_ans_rep = list(zip(test1_probe, test1_ans, test1_order, pair_idx))
    probe_ans_rep = rng.permutation(probe_ans_rep)
    test1_probe, test1_ans, test1_order, test1_idx = zip(*probe_ans_rep)
    test1_probe, test1_ans, test1_order, test1_idx = np.array(test1_probe), np.array(test1_ans), np.array(test1_order), np.array(test1_idx)
    
    tmp_df_test1['test_itemno1'] = test1_probe
    tmp_df_test1['test_itemno2'] = -1
    tmp_df_test1['test_item'] = items[test1_probe-1]
    tmp_df_test1['correct_ans'] = test1_ans
    tmp_df_test1['order'] = test1_order
    tmp_df_test1['rep'] = 1
    tmp_df_test1['test'] = 1
    tmp_df_test1['session'] = sess
    tmp_df_test1['test_pos'] = np.arange(1,13)
    tmp_df_test1['pair_idx'] = test1_idx

    probe_ans_rep = list(zip(test2_probe, test2_ans, test2_order, pair_idx))
    probe_ans_rep = rng.permutation(probe_ans_rep)
    test2_probe, test2_ans, test2_order, test2_idx = zip(*probe_ans_rep)
    test2_probe, test2_ans, test2_order, test2_idx = np.array(test2_probe), np.array(test2_ans), np.array(test2_order), np.array(test2_idx)

    tmp_df_test2['test_itemno1'] = test2_probe
    tmp_df_test2['test_itemno2'] = -1
    tmp_df_test2['test_item'] = items[test2_probe-1]
    tmp_df_test2['correct_ans'] = test2_ans
    tmp_df_test2['order'] = test2_order
    tmp_df_test2['rep'] = 1
    tmp_df_test2['test'] = 2
    tmp_df_test2['session'] = sess
    tmp_df_test2['test_pos'] = np.arange(13,25)
    tmp_df_test2['pair_idx'] = test2_idx
    df_test = pd.concat([df_test,tmp_df_test1, tmp_df_test2])

df_study.reset_index(inplace = True, drop=True)
df_test.reset_index(inplace = True, drop=True)  

In [12]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session
0,140,838,BLUEBERRY,LOCK,0
1,1587,1240,WARDROBE,SAXOPHONE,0
2,1132,702,PROTON,HELMET,0
3,299,1282,CHILD,SHELTER,0
4,1391,1059,STRAWBERRY,PICKLE,0
...,...,...,...,...,...
11995,301,1184,CHIMNEY,REPORT,999
11996,57,965,ATTIRE,NUT,999
11997,155,1112,BOULDER,POSSUM,999
11998,106,1368,BEAR,STAKE,999


In [13]:
df_test.head(48)

,test_itemno1,test_itemno2,test_item,correct_ans,order,rep,test,session,test_pos,pair_idx
0,210,-1,CABBAGE,327,1,1,1,0,1,11
1,1251,-1,SCOTCH,819,2,1,1,0,2,8
2,145,-1,BODY,1198,1,1,1,0,3,7
3,1059,-1,PICKLE,1391,2,1,1,0,4,5
4,711,-1,HOE,1519,1,1,1,0,5,9
5,838,-1,LOCK,140,2,1,1,0,6,1
6,153,-1,BOTTLE,736,1,1,1,0,7,6
7,1587,-1,WARDROBE,1240,1,1,1,0,8,2
8,856,-1,MAID,607,2,1,1,0,9,10
9,702,-1,HELMET,1132,2,1,1,0,10,3


In [14]:
# save
with open('../Data/simu6c_design.pkl', 'wb') as outp:
    pickle.dump(df_study, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(df_test, outp, pickle.HIGHEST_PROTOCOL)